# Проект e-learning: вариант 2
Продакт-менеджер Василий попросил вас проанализировать завершенные уроки и ответить на следующие вопросы:

1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене) (7 баллов).

2. Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью*. (5 баллов)

3. По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом). (5 баллов) 

4. Выяви самые популярные курсы (ТОП-3) по количеству регистраций на них. А также курсы с самым большим оттоком (ТОП-3). (8 баллов)

5. Напиши функцию на python, позволяющую строить когортный (семестровый) анализ. В период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов. Описание подхода можно найти тут. (10 баллов) 

6. Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можешь выбрать следующие метрики: R - среднее время сдачи одного экзамена, F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 0.1 до 0.5, monetary от 55 до 72 баллов. Описание подхода можно найти тут. (35 баллов)

*завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

Файлы: 

**assessments.csv**  — этот файл содержит информацию об оценках в тесте. Обычно каждый предмет в семестре включает ряд тестов с оценками, за которыми следует заключительный экзаменационный тест (экзамен).
* code_module — идентификационный код предмета.
* code_presentation — семестр (Идентификационный код).
* id_assessment — тест (Идентификационный номер ассессмента).
* assessment_type — тип теста. Существуют три типа оценивания: оценка преподавателя (TMA), компьютерная оценка (СМА), экзамен по курсу (Exam).
* date — информация об окончательной дате сдачи теста. Рассчитывается как количество дней с момента начала семестра. Дата начала семестра имеет номер 0 (ноль).
* weight — вес теста в % в оценке за курс. Обычно экзамены рассматриваются отдельно и имеют вес 100%; сумма всех остальных оценок составляет 100%.

**courses.csv** — файл содержит список предметов по семестрам.
* code_module — предмет (идентификационный код).
* code_presentation — семестр (идентификационный код).
* module_presentation_length — продолжительность семестра в днях.

**studentAssessment.csv** — этот файл содержит результаты тестов студентов. Если учащийся не отправляет работу на оценку, результат не записывается в таблицу.
* id_assessment — тест (идентификационный номер).
* id_student — идентификационный номер студента.
* date_submitted — дата сдачи теста студентом, измеряемая как количество дней с начала семестра.
* is_banked — факт перезачета теста с прошлого семестра (иногда курсы перезачитывают студентам, вернувшимся из академического отпуска).
* score — оценка учащегося в этом тесте. Диапазон составляет от 0 до 100. Оценка ниже 40 неудачная/неуспешная сдача теста.

**studentRegistration.csv** — этот файл содержит информацию о времени, когда студент зарегистрировался для прохождения курса в семестре.
* code_module — предмет (идентификационный код).
* code_presentation — семестр (идентификационный код)
* id_student — идентификационный номер студента.
* date_registration — дата регистрации студента. Это количество дней, измеренное от начала семестра (например, отрицательное значение -30 означает, что студент зарегистрировался на прохождение курса за 30 дней до его начала).
* date_unregistration — дата отмены регистрации студента с предмета. У студентов, окончивших курс, это поле остается пустым.

In [52]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import plotly.express as px
from io import BytesIO
import requests

In [53]:
sns.set(
    font_scale =2,
    style      ="whitegrid",
    rc         ={'figure.figsize':(60,10)},
)

## Задание 1
Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене) (7 баллов).

In [54]:
path_to_file = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-e-lapin/' +\
               'Projects/first_project/Files/assessments.csv'
assessments = pd.read_csv(path_to_file)
assessments

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0
...,...,...,...,...,...,...
201,GGG,2014J,37443,CMA,229.0,0.0
202,GGG,2014J,37435,TMA,61.0,0.0
203,GGG,2014J,37436,TMA,124.0,0.0
204,GGG,2014J,37437,TMA,173.0,0.0


In [55]:
path_to_file = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-e-lapin/' +\
                'Projects/first_project/Files/studentAssessment.csv'
studentAssessment = pd.read_csv(path_to_file)
studentAssessment

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0
...,...,...,...,...,...
173907,37443,527538,227,0,60.0
173908,37443,534672,229,0,100.0
173909,37443,546286,215,0,80.0
173910,37443,546724,230,0,100.0


In [56]:
results = studentAssessment.merge(assessments, how='left', on='id_assessment')
results

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0
...,...,...,...,...,...,...,...,...,...,...
173907,37443,527538,227,0,60.0,GGG,2014J,CMA,229.0,0.0
173908,37443,534672,229,0,100.0,GGG,2014J,CMA,229.0,0.0
173909,37443,546286,215,0,80.0,GGG,2014J,CMA,229.0,0.0
173910,37443,546724,230,0,100.0,GGG,2014J,CMA,229.0,0.0


In [57]:
results = results.dropna(subset=['score']) # дроп строк с пустым значением в столбце оценка,
                                           # в дальнейшем с ними работать не будем

In [58]:
results['is_suc'] = np.where(results['score']  >= 40, 1, 0)
results

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,is_suc
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0,1
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0,1
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0,1
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0,1
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0,1
...,...,...,...,...,...,...,...,...,...,...,...
173907,37443,527538,227,0,60.0,GGG,2014J,CMA,229.0,0.0,1
173908,37443,534672,229,0,100.0,GGG,2014J,CMA,229.0,0.0,1
173909,37443,546286,215,0,80.0,GGG,2014J,CMA,229.0,0.0,1
173910,37443,546724,230,0,100.0,GGG,2014J,CMA,229.0,0.0,1


In [59]:
# кол-во студентов с 1 сданным курсом
results\
            .query('assessment_type == "Exam" and is_suc == 1')\
            .id_student\
            .value_counts()\
            .loc[lambda x: x == 1]\
            .count()

3802

In [219]:
# тесты с экзаменами
results.query('assessment_type == "Exam"').id_assessment.value_counts()

24299    1168
25354     968
25368     950
24290     747
25340     602
25361     524
Name: id_assessment, dtype: int64

## Задание 2
Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью*. (5 баллов)

In [85]:
# скопируем таблицу для удобства работы
completion = results
completion

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,is_suc
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0,1
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0,1
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0,1
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0,1
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0,1
...,...,...,...,...,...,...,...,...,...,...,...
173907,37443,527538,227,0,60.0,GGG,2014J,CMA,229.0,0.0,1
173908,37443,534672,229,0,100.0,GGG,2014J,CMA,229.0,0.0,1
173909,37443,546286,215,0,80.0,GGG,2014J,CMA,229.0,0.0,1
173910,37443,546724,230,0,100.0,GGG,2014J,CMA,229.0,0.0,1


In [86]:
# проверяем были ли среди студентов те, кто сдавал экзамен несколько раз, но с разными результатами
completion.loc[completion.duplicated(subset=['id_student','id_assessment'])] 

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,is_suc


In [87]:
# замечаем, что один предмет мог быть в нескольких семестрах
completion\
                .query('assessment_type == "Exam"')\
                .groupby(['id_assessment','is_suc','code_module','code_presentation'], as_index=False)\
                .agg({'assessment_type': 'count'})\
                .rename(columns={'assessment_type':'amount'})

,id_assessment,is_suc,code_module,code_presentation,amount
0,24290,0,CCC,2014B,83
1,24290,1,CCC,2014B,664
2,24299,0,CCC,2014J,149
3,24299,1,CCC,2014J,1019
4,25340,0,DDD,2013B,98
5,25340,1,DDD,2013B,504
6,25354,0,DDD,2013J,90
7,25354,1,DDD,2013J,878
8,25361,0,DDD,2014B,39
9,25361,1,DDD,2014B,485


In [88]:
# группировка по экзамену
exam_completion = completion\
                .query('assessment_type == "Exam"')\
                .groupby(['id_assessment','is_suc'], as_index=False)\
                .agg({'assessment_type': 'count'})\
                .rename(columns={'assessment_type':'amount'})

exam_completion

,id_assessment,is_suc,amount
0,24290,0,83
1,24290,1,664
2,24299,0,149
3,24299,1,1019
4,25340,0,98
5,25340,1,504
6,25354,0,90
7,25354,1,878
8,25361,0,39
9,25361,1,485


In [89]:
exam_completion_pivot = exam_completion.pivot(index='id_assessment', columns='is_suc', values='amount').reset_index()
exam_completion_pivot

is_suc,id_assessment,0,1
0,24290,83,664
1,24299,149,1019
2,25340,98,504
3,25354,90,878
4,25361,39,485
5,25368,108,842


In [90]:
exam_completion_pivot.columns = ['id_assessment', 'not_passed', 'passed']

In [91]:
exam_completion_pivot['passed_percent'] = exam_completion_pivot.passed /\
                                         (exam_completion_pivot.passed + exam_completion_pivot.not_passed) * 100
exam_completion_pivot

,id_assessment,not_passed,passed,passed_percent
0,24290,83,664,88.888889
1,24299,149,1019,87.243151
2,25340,98,504,83.720930
3,25354,90,878,90.702479
4,25361,39,485,92.557252
5,25368,108,842,88.631579


In [92]:
# самый легкий экзамен
exam_completion_pivot.loc[exam_completion_pivot['passed_percent'].idxmax(),'id_assessment']

25361

In [93]:
# самый сложный экзамен
exam_completion_pivot.loc[exam_completion_pivot['passed_percent'].idxmin(),'id_assessment']

25340

In [94]:
# замечаем, что один предмет мог быть в нескольких семестрах
module_completion = completion\
                .query('assessment_type == "Exam"')\
                .groupby(['code_module','is_suc'], as_index=False)\
                .agg({'assessment_type': 'count'})\
                .rename(columns={'assessment_type':'amount'})

module_completion

,code_module,is_suc,amount
0,CCC,0,232
1,CCC,1,1683
2,DDD,0,335
3,DDD,1,2709


In [95]:
module_completion_pivot = module_completion.pivot(index='code_module', columns='is_suc', values='amount')\
                                           .reset_index()
module_completion_pivot

is_suc,code_module,0,1
0,CCC,232,1683
1,DDD,335,2709


In [96]:
module_completion_pivot.columns = ['id_assessment', 'not_passed', 'passed']

In [97]:
module_completion_pivot['passed_percent'] = module_completion_pivot.passed /\
                                         (module_completion_pivot.passed + module_completion_pivot.not_passed) * 100
module_completion_pivot

,id_assessment,not_passed,passed,passed_percent
0,CCC,232,1683,87.885117
1,DDD,335,2709,88.994744


In [98]:
# самый легкий предмет по коду
module_completion_pivot.loc[module_completion_pivot['passed_percent'].idxmax(),'id_assessment']

'DDD'

In [99]:
# самый сложный предмет по коду
module_completion_pivot.loc[module_completion_pivot['passed_percent'].idxmin(),'id_assessment']

'CCC'

## Задание 3
По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом). (5 баллов)

In [356]:
# по id теста
completion\
            .query('assessment_type == "Exam" and is_suc == 1')\
            .groupby(['id_assessment'],as_index=False )\
            .agg({'date_submitted':'mean'})\
            .rename(columns={'date_submitted':'avg_date_submitted'})

,id_assessment,avg_date_submitted
0,24290,231.581325
1,24299,244.404318
2,25340,230.164683
3,25354,239.509112
4,25361,234.936082
5,25368,242.804038


In [357]:
# по коду предмета
completion\
            .query('assessment_type == "Exam" and is_suc == 1')\
            .groupby(['code_module'],as_index=False )\
            .agg({'date_submitted':'mean'})\
            .rename(columns={'date_submitted':'avg_date_submitted'})

,code_module,avg_date_submitted
0,CCC,239.345217
1,DDD,237.976006


## Задание 4
Выяви самые популярные курсы (ТОП-3) по количеству регистраций на них. А также курсы с самым большим оттоком (ТОП-3). (8 баллов)

In [221]:
path_to_file = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-e-lapin/' +\
                'Projects/first_project/Files/studentRegistration.csv'
studentRegistration = pd.read_csv(path_to_file)
studentRegistration

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN
...,...,...,...,...,...
32588,GGG,2014J,2640965,-4.0,NaN
32589,GGG,2014J,2645731,-23.0,NaN
32590,GGG,2014J,2648187,-129.0,NaN
32591,GGG,2014J,2679821,-49.0,101.0


In [233]:
# самые популярные курсы (модули) (ТОП-3) по количеству регистраций
studentRegistration.code_module.value_counts().head(3)

BBB    7909
FFF    7762
DDD    6272
Name: code_module, dtype: int64

In [276]:
# курсы (модули) с самым большим оттоком (ТОП-3)
studentRegistration\
                    .dropna(subset=['date_unregistration'])\
                    .groupby(['code_module'], as_index=False)\
                    .agg({'date_unregistration':'count'})\
                    .sort_values('date_unregistration', ascending=False)\
                    .head(3)

,code_module,date_unregistration
5,FFF,2380
1,BBB,2377
3,DDD,2235


### Если смотреть по id теста
В данном варианте некоторых модулей не будет, так как для них не было типа теста экзамен, либо не стояла оценка

In [227]:
code_with_id_assessment = studentRegistration.merge(results, how='left', on='id_student') # объединим таблицы по id студентов
code_with_id_assessment = code_with_id_assessment.query('assessment_type == "Exam"') # отсеем обычные тесты
code_with_id_assessment

,code_module_x,code_presentation_x,id_student,date_registration,date_unregistration,id_assessment,date_submitted,is_banked,score,code_module_y,code_presentation_y,assessment_type,date,weight,is_suc
13921,BBB,2013B,560458,-21.0,-2.0,25354.0,241.0,0.0,73.0,DDD,2013J,Exam,261.0,100.0,1.0
20215,BBB,2013J,523857,-100.0,NaN,25361.0,234.0,0.0,56.0,DDD,2014B,Exam,241.0,100.0,1.0
44035,BBB,2014J,583775,-41.0,NaN,25361.0,234.0,0.0,71.0,DDD,2014B,Exam,241.0,100.0,1.0
44868,BBB,2014J,609194,-162.0,NaN,25361.0,234.0,0.0,73.0,DDD,2014B,Exam,241.0,100.0,1.0
50272,CCC,2014B,29764,-34.0,NaN,24290.0,230.0,0.0,94.0,CCC,2014B,Exam,NaN,100.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196531,FFF,2014J,2499307,-23.0,NaN,25340.0,231.0,0.0,20.0,DDD,2013B,Exam,240.0,100.0,0.0
196690,FFF,2014J,2559144,-52.0,NaN,24299.0,245.0,0.0,98.0,CCC,2014J,Exam,NaN,100.0,1.0
196764,FFF,2014J,2599883,-72.0,NaN,24299.0,244.0,0.0,96.0,CCC,2014J,Exam,NaN,100.0,1.0
197450,GGG,2013J,339853,-67.0,NaN,25368.0,242.0,0.0,29.0,DDD,2014J,Exam,NaN,100.0,0.0


In [231]:
# самые популярные курсы (тесты) (ТОП-3) по количеству регистраций
code_with_id_assessment.id_assessment.value_counts().head(3)

24299.0    1812
25354.0    1246
25368.0    1108
Name: id_assessment, dtype: int64

In [235]:
# курсы (модули) с самым большим оттоком (ТОП-3)
code_with_id_assessment\
                    .dropna(subset=['date_unregistration'])\
                    .groupby(['id_assessment'], as_index=False)\
                    .agg({'date_unregistration':'count'})\
                    .sort_values('date_unregistration', ascending=False)\
                    .head(3)

,id_assessment,date_unregistration
3,25354.0,113
5,25368.0,92
1,24299.0,89


## Задание 5
Напиши функцию на python, позволяющую строить когортный (семестровый) анализ. В период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов. Описание подхода можно найти тут. (10 баллов) https://vc.ru/s/productstar/134090-chto-takoe-kogortnyy-analiz

In [100]:
def cohort(table):
    # создаем первую таблицу, отбирая семестры с тестами, у которых имеются сданные экзамены
    first_table = table\
                    .query('assessment_type == "Exam"')\
                    .groupby(['code_presentation','is_suc'], as_index=False)\
                    .agg({'assessment_type': 'count'})\
                    .rename(columns={'assessment_type':'amount'})
    # используем pivot для изменения вида
    first_table = first_table.pivot(index='code_presentation', columns='is_suc', values='amount').reset_index()
    # переименовываем столбцы
    first_table.columns = ['code_presentation', 'not_passed','passed']
    # считаем процент успешных экзаменов
    first_table['passed_percent'] = first_table.passed /\
                                         (first_table.passed + first_table.not_passed) * 100
    # создаем вторую таблицу, получая средние для каждого семестра
    second_table = table\
                .query('assessment_type == "Exam" and is_suc == 1')\
                .groupby(['code_presentation'],as_index=False )\
                .agg({'date_submitted':'mean'})\
                .rename(columns={'date_submitted':'avg_date_submitted'})
    # объединяем таблицы
    final_table = first_table.merge(second_table, how='left', on='code_presentation')
    final_table = final_table[['code_presentation','avg_date_submitted','passed_percent']]
    final_table.columns = ['semester', 'avg_date_submitted', 'passed_percent']
 
    return final_table

In [101]:
# применяем функцию
cohort_table = cohort(completion)
cohort_table

,semester,avg_date_submitted,passed_percent
0,2013B,230.164683,83.720930
1,2013J,239.509112,90.702479
2,2014B,232.997389,90.401259
3,2014J,243.680279,87.865911


In [102]:
print('Семестр с самой низкой завершаемостью курсов: ',\
      cohort_table.loc[cohort_table['passed_percent'].idxmin(),'semester'])
print('Cеместр самыми долгими средними сроками сдачи курсов: ',\
      cohort_table.loc[cohort_table['avg_date_submitted'].idxmax(),'semester'])

Семестр с самой низкой завершаемостью курсов:  2013B
Cеместр самыми долгими средними сроками сдачи курсов:  2014J


## Задание 6
Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. 

В адаптированной кластеризации можешь выбрать следующие метрики: 
* R - среднее время сдачи одного экзамена, 
* F - завершаемость курсов, 
* M - среднее количество баллов, получаемое за экзамен. 

Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик `recency`, `frequency` и `monetary` для интерпретации этих кластеров. 

Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 0.1 до 0.5, monetary от 55 до 72 баллов. Описание подхода можно найти тут.(35 баллов)
https://medianation.ru/blog/analitika/avtomatizatsiya-rfm-analiza-kak-segmentirovat-klientskuyu-bazu-na-python/

Будем использовать предложенные в задании следующие метрики:
* R - среднее время сдачи одного экзамена,
* F - завершаемость курсов,
* M - среднее количество баллов, получаемое за экзамен.

В данном задании под курсом будем понимать = тест с типом экзамен

In [207]:
# R - среднее время сдачи одного экзамена
# В данной таблице меньше значений, т.к. под сдачей понимаем последнее успешное прохождение экзамена студентом, часть студентов
# не сдало ни 1 экзамена
student_r = completion\
            .query('assessment_type == "Exam" and is_suc == 1')\
            .groupby(['id_student'],as_index=False )\
            .agg({'date_submitted':'mean'})\
            .rename(columns={'date_submitted':'avg_date_submitted'})

student_r

,id_student,avg_date_submitted
0,23698,243.0
1,24213,236.0
2,27116,243.0
3,28046,237.0
4,28787,243.0
...,...,...
4092,2694886,236.0
4093,2694933,230.0
4094,2695608,237.0
4095,2697181,230.0


In [208]:
# найдем моду для R. Поскольку часть людей успешно не проходила ни один экзамен, то нагляднее будет не строить первый промежуток
# в виде, например, 0-40, а взять точку 0. А для разделения оставшегося промежутка использовать медиану
# итого [0, > 0 and <= 240, > 240]
student_r.avg_date_submitted.median()

241.0

In [209]:
# F - завершаемость курсов
student_f = completion\
                .query('assessment_type == "Exam"')\
                .groupby(['id_student','is_suc'], as_index=False)\
                .agg({'assessment_type': 'count'})\
                .rename(columns={'assessment_type':'amount'})\
                .pivot(index='id_student', columns='is_suc', values='amount')\
                .reset_index()
    
student_f.columns = ['id_student', 'not_passed', 'passed']
student_f = student_f.fillna(0) # заменим пустые значения для дальнейших вычислений 

student_f['passed_percent'] = student_f.passed / (student_f.passed + student_f.not_passed) * 100
student_f

,id_student,not_passed,passed,passed_percent
0,23698,0.0,1.0,100.0
1,24213,0.0,1.0,100.0
2,27116,0.0,1.0,100.0
3,28046,0.0,1.0,100.0
4,28787,0.0,1.0,100.0
...,...,...,...,...
4628,2694886,0.0,1.0,100.0
4629,2694933,0.0,1.0,100.0
4630,2695608,0.0,1.0,100.0
4631,2697181,0.0,1.0,100.0


In [210]:
# M - среднее количество баллов, получаемое за экзамен
student_m = completion\
            .query('assessment_type == "Exam"')\
            .groupby(['id_student'],as_index=False )\
            .agg({'score':'mean'})\
            .rename(columns={'score':'avg_score'})

student_m

,id_student,avg_score
0,23698,80.0
1,24213,58.0
2,27116,96.0
3,28046,40.0
4,28787,44.0
...,...,...
4628,2694886,69.0
4629,2694933,73.0
4630,2695608,73.0
4631,2697181,80.0


In [211]:
# собираем результирующую таблицу
student_f_m = student_f.merge(student_m, how='left', on='id_student')
student_rfm = student_f_m.merge(student_r, how='left', on='id_student')
student_rfm = student_rfm.fillna(0) # 0 в столбце avg_date_submitted будет означать, что студент так и не сдал экзамен
student_rfm = student_rfm[['id_student','avg_date_submitted','passed_percent','avg_score']]
student_rfm = student_rfm.rename(columns={'avg_date_submitted':'r_avg_date_submitted',\
                            'passed_percent':'f_passed_percent', \
                            'avg_score':'m_avg_score'})
student_rfm

,id_student,r_avg_date_submitted,f_passed_percent,m_avg_score
0,23698,243.0,100.0,80.0
1,24213,236.0,100.0,58.0
2,27116,243.0,100.0,96.0
3,28046,237.0,100.0,40.0
4,28787,243.0,100.0,44.0
...,...,...,...,...
4628,2694886,236.0,100.0,69.0
4629,2694933,230.0,100.0,73.0
4630,2695608,237.0,100.0,73.0
4631,2697181,230.0,100.0,80.0


In [212]:
# описание значений параметра F
student_rfm.f_passed_percent.describe()

count    4633.000000
mean       88.128642
std        32.114175
min         0.000000
25%       100.000000
50%       100.000000
75%       100.000000
max       100.000000
Name: f_passed_percent, dtype: float64

In [213]:
# в столбце встречаются лишь три значения: 0,50,100. Можно взять данные значения для границ frequency в качестве точек, однако
# это частный случай, поэтому граница 50 выбрана на основе значений, а вторая граница 88 относительно среднего - mean 
# итого: [< 50, >=50 and < 88, >= 88]
student_rfm.f_passed_percent.value_counts()

100.0    4069
0.0       536
50.0       28
Name: f_passed_percent, dtype: int64

In [214]:
# первая граница выбрана на основе порога успешности сдачи экзамена <40, 
# вторая граница выбрана на основе среднего значения в столбце - 65
# итого: [< 40, >=40 and < 65, >= 65]
student_rfm.m_avg_score.describe()

count    4633.000000
mean       65.117958
std        20.470561
min         0.000000
25%        50.000000
50%        66.000000
75%        82.000000
max       100.000000
Name: m_avg_score, dtype: float64

In [216]:
# обозначим границы каждой из метрик
df = pd.DataFrame({"R": ['0 дней', 'от 0 до 240 дней включительно', 'больше 240 дней'],\
                   "F": ['меньше 50 %', 'от 50 до 88 %', 'больше или равно 88 %'],\
                   "M": ['меньше 40 баллов', 'от 40 до 65 баллов', 'больше или равно 65 баллов']})
df

,R,F,M
0,0 дней,меньше 50 %,меньше 40 баллов
1,от 0 до 240 дней включительно,от 50 до 88 %,от 40 до 65 баллов
2,больше 240 дней,больше или равно 88 %,больше или равно 65 баллов


In [217]:
#описание RFM сегментов
r = ['0 дней', 'от 0 до 240 дней включительно', 'больше 240 дней']
f = ['меньше 50 %', 'от 50 до 88 %', 'больше или равно 88 %']
m = ['меньше 40 баллов', 'от 40 до 65 баллов', 'больше или равно 65 баллов']

for i in range(len(r)):
    for j in range(len(f)):  
        for k in range(len(m)):
            rfm = 'RFM-сегмент '+ str(i+1) + str(j+1) + str(k+1) + \
                  ' recency=' + str(i+1) + ' frequency=' + str(j+1) + ' monetary=' + str(k+1) + \
                  ', имеет границы метрик: recency ' + r[i] + ', frequency ' + f[j] + ', monetary '+ m[k]
            print(rfm)

RFM-сегмент 111 recency=1 frequency=1 monetary=1, имеет границы метрик: recency 0 дней, frequency меньше 50 %, monetary меньше 40 баллов
RFM-сегмент 112 recency=1 frequency=1 monetary=2, имеет границы метрик: recency 0 дней, frequency меньше 50 %, monetary от 40 до 65 баллов
RFM-сегмент 113 recency=1 frequency=1 monetary=3, имеет границы метрик: recency 0 дней, frequency меньше 50 %, monetary больше или равно 65 баллов
RFM-сегмент 121 recency=1 frequency=2 monetary=1, имеет границы метрик: recency 0 дней, frequency от 50 до 88 %, monetary меньше 40 баллов
RFM-сегмент 122 recency=1 frequency=2 monetary=2, имеет границы метрик: recency 0 дней, frequency от 50 до 88 %, monetary от 40 до 65 баллов
RFM-сегмент 123 recency=1 frequency=2 monetary=3, имеет границы метрик: recency 0 дней, frequency от 50 до 88 %, monetary больше или равно 65 баллов
RFM-сегмент 131 recency=1 frequency=3 monetary=1, имеет границы метрик: recency 0 дней, frequency больше или равно 88 %, monetary меньше 40 баллов
R

In [218]:
# переприсвоим таблицу для удобства работы
student_rfm_final = student_rfm

# создадим новые столбцы для RFM анализа, используем pd.cut в соответствии с границами метрик
student_rfm_final['r'] = pd.cut(student_rfm.r_avg_date_submitted, [-1,0,240,100000] ,\
                                labels=['1','2','3'])
student_rfm_final['f'] = pd.cut(student_rfm.f_passed_percent, [0,50,88,101] ,\
                                labels=['1','2','3'], right=False)
student_rfm_final['m'] = pd.cut(student_rfm.m_avg_score, [0,40,65,101] ,\
                                labels=['1','2','3'], right=False)
student_rfm_final = student_rfm_final[['id_student','r','f','m']]
student_rfm_final

,id_student,r,f,m
0,23698,3,3,3
1,24213,2,3,2
2,27116,3,3,3
3,28046,2,3,2
4,28787,3,3,2
...,...,...,...,...
4628,2694886,2,3,3
4629,2694933,2,3,3
4630,2695608,2,3,3
4631,2697181,2,3,3
